### Wangsu Hu
### 2018 02 08 
### after collection the firehose data, this time i will use pandas to clean the dataset
### the dataset for testing is only 100, later i will try the full 1 million data

In [1]:
cd C:\Devel

C:\Devel


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### only do 100 data

In [3]:
n=100

In [4]:
df1=pd.read_csv('./firehose.csv',
                usecols=['ROWNUM','LAT','LNG','COLLECT_TIME','TIMEZONEOFFSET'],
                nrows=n)
df2=pd.read_csv('./firehose.csv',usecols=['CATEGORY'],nrows=n)

### some category id will contain '[' there, i have to remove it

In [5]:
for i in tqdm(range(len(df2))):
    cate=str(df2.loc[i][0])
    if cate[0]=='[':
        df2.loc[i][0]=cate[1:]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 4344.85it/s]


In [6]:
df=df1.join(df2, how='outer')

In [7]:
cd C:\Users\Nicholas\Dropbox\2018.02.28 firehosedata process

C:\Users\Nicholas\Dropbox\2018.02.28 firehosedata process


### this tells how to collect the foursquare hierarchical categorization. the return file is in json format, we need to re-constructe it

In [8]:
import foursquare

import json

# obtained from foursquare API home by registering a client
client_id = 'your id'
client_secret = 'your psd'

client = foursquare.Foursquare(client_id=client_id, client_secret=client_secret)

out = open('foursquare-categories.json', 'w')
out.write(json.dumps(client.venues.categories(), sort_keys=True, indent=4))
out.close()

In [9]:
filename='foursquare-categories.json'

with open(filename, 'r') as f:
        datastore = json.load(f)

### the following code is for specific use of certain level of category, as i know, foursquare contain at leaset five level. For my research, I only map the first level (10 types)

In [10]:
first_cate=[[datastore['categories'][i]['id'],datastore['categories'][i]['name']] for i in range(len(datastore['categories']))]

In [11]:
sec_cate=[[datastore['categories'][i]['id'],
           datastore['categories'][i]['name'],
           [[datastore['categories'][i]['categories'][j]['id'], 
            datastore['categories'][i]['categories'][j]['name']]  
           for j in range(len(datastore['categories'][i]['categories']))] 
          ]
for i in range(len(datastore['categories']))]

In [12]:
third_cate=[[datastore['categories'][i]['id'],
           datastore['categories'][i]['name'],
           [[datastore['categories'][i]['categories'][j]['id'], 
            datastore['categories'][i]['categories'][j]['name'],
            [[datastore['categories'][i]['categories'][j]['categories'][k]['id'],
              datastore['categories'][i]['categories'][j]['categories'][k]['name']] 
             for k in range(len(datastore['categories'][i]['categories'][j]['categories']))]
            ]  
           for j in range(len(datastore['categories'][i]['categories']))] 
          ]
for i in range(len(datastore['categories']))]

### this is what i really want

In [13]:
map={}
for i in range(len(datastore['categories'])):
    map[datastore['categories'][i]['id']]=datastore['categories'][i]['name']
    for j in range(len(datastore['categories'][i]['categories'])):
        map[datastore['categories'][i]['categories'][j]['id']]=datastore['categories'][i]['name']
        for k in range(len(datastore['categories'][i]['categories'][j]['categories'])):
            map[datastore['categories'][i]['categories'][j]['categories'][k]['id']]=datastore['categories'][i]['name']
            for l in range(len(datastore['categories'][i]['categories'][j]['categories'][k]['categories'])):
                map[datastore['categories'][i]['categories'][j]['categories'][k]['categories'][l]['id']]=datastore['categories'][i]['name']
                for m in range(len(datastore['categories'][i]['categories'][j]['categories'][k]['categories'][l]['categories'])):
                    map[datastore['categories'][i]['categories'][j]['categories'][k]['categories'][l]['categories'][m]['id']]=datastore['categories'][i]['name']
                    for n in range(len(datastore['categories'][i]['categories'][j]['categories'][k]['categories'][l]['categories'][m]['categories'])):
                        map[datastore['categories'][i]['categories'][j]['categories'][k]['categories'][l]['categories'][m]['categories'][n]['id']]=datastore['categories'][i]['name']

In [14]:
newcol = df['CATEGORY']
df=df.assign(CATEGORY_NAME=newcol)

In [15]:
df.dtypes

ROWNUM              int64
LAT               float64
LNG               float64
COLLECT_TIME        int64
TIMEZONEOFFSET      int64
CATEGORY           object
CATEGORY_NAME      object
dtype: object

In [16]:
for i in tqdm(range(len(df))):
    cid=df['CATEGORY'][i]
    if cid in map:
        df.set_value(i, 'CATEGORY_NAME', map[cid])

  0%|                                                  | 0/100 [00:00<?, ?it/s]C:\ProgramData\Anaconda2\envs\tf\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
100%|█████████████████████████████████████| 100/100 [00:00<00:00, 28542.39it/s]


### now the category name is there

In [17]:
df

,ROWNUM,LAT,LNG,COLLECT_TIME,TIMEZONEOFFSET,CATEGORY,CATEGORY_NAME
0,1,40.340646,36.562329,1472501532,180,4bf58dd8d48988d165941735,Outdoors & Recreation
1,2,37.036869,35.314641,1472501532,180,4bf58dd8d48988d116941735,Nightlife Spot
2,3,39.598118,26.973218,1472501532,180,4bf58dd8d48988d1e0931735,Food
3,4,38.388492,27.165315,1472501532,180,4bf58dd8d48988d116941735,Nightlife Spot
4,5,36.754580,34.538092,1472501532,180,4bf58dd8d48988d1d0941735,Food
5,6,40.353397,27.969922,1472501532,180,4bf58dd8d48988d16d941735,Food
6,7,43.977432,25.336375,1472501533,180,4bf58dd8d48988d162941735,Outdoors & Recreation
7,8,41.732030,27.214883,1472501532,180,4bf58dd8d48988d16d941735,Food
8,9,41.016194,28.591746,1472501532,180,4bf58dd8d48988d16d941735,Food
9,10,38.614560,27.347446,1472501532,180,4bf58dd8d48988d15a941735,Outdoors & Recreation
